In [32]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [33]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

In [34]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [35]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_90.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 122 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(122): SRR7817613 SRR7817625 ... SRR7817630 SRR7817679
colData names(3): Sample Dose sizeFactor


In [36]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [37]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 122
                   SRR7817613 SRR7817625 SRR3593523 SRR7817676 SRR4317609
ENSMUSG00000000001  15.678920  15.794204  14.802368  15.792070  15.257084
ENSMUSG00000000028   8.373881   8.010359   8.129061   8.402368   8.704348
ENSMUSG00000000049  20.478337  20.313465  20.754121  20.038138  20.600894
ENSMUSG00000000058   8.871176   8.801971   9.197780   9.541066   9.859166
ENSMUSG00000000085  10.969441  11.280353  11.256788  10.338241  10.352565
ENSMUSG00000000088  16.818565  16.528197  15.272027  16.605132  14.719661
                   SRR7817660 SRR7817664 SRR7817648 SRR7817698 SRR7817651
ENSMUSG00000000001  15.856177  15.530593  15.446439  15.544915  15.813822
ENSMUSG00000000028   8.303532   8.032180   8.124688   7.973644   8.602236
ENSMUSG00000000049  20.164105  20.345501  20.527072  20.309176  20.190658
ENSMUSG00000000058   8.975952   9.078611   8.706991   8.868303   8.983531
ENSMUSG00000000085  11.046926  10.509282  10.980802  10.712587  10.772729
ENSMUSG00000000088  16.217968 

ENSMUSG00000000085  11.334053  11.406258  11.077985  11.231316  10.554823
ENSMUSG00000000088  16.210264  16.396966  16.672032  15.730048  15.595355
                   SRR4317656 SRR7817653 SRR1636676 SRR3593580 SRR7817658
ENSMUSG00000000001  15.989159  16.210722   16.09063  15.728697  15.726745
ENSMUSG00000000028   9.270205   9.016255    8.99182   8.301539   9.070129
ENSMUSG00000000049  17.520469  17.761444   19.91709  18.966030  18.133456
ENSMUSG00000000058  10.962863   9.870056   10.46059   9.918868   9.551293
ENSMUSG00000000085  10.919170  11.028928   10.69986  11.050336  11.028603
ENSMUSG00000000088  14.639334  16.385458   16.16076  15.056770  16.125089
                   SRR7817632 SRR4317660 SRR7817669 SRR1636671 SRR3593578
ENSMUSG00000000001  16.646082  15.745265  15.892938  16.600188  15.756063
ENSMUSG00000000028   8.732025   8.928464   9.154559   8.571282   8.290847
ENSMUSG00000000049  16.988772  17.705794  18.237654  19.849412  18.957115
ENSMUSG00000000058   9.965348  10.0949

In [38]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_90.Rds", sep = ""))
print(tcdd_dose_detail.vec)

  [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [51]  0  0  0  0  0  0  0  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 [76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[101] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [39]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_90.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





$`R-MMU-9794270`
[1] "ENSMUSG00000030237"

$`R-MMU-70510`
[1] "ENSMUSG00000030246" "ENSMUSG00000030851" "ENSMUSG00000063229"
[4] "ENSMUSG00000101959"

$`R-MMU-71849`
[1] "ENSMUSG00000030246" "ENSMUSG00000030851" "ENSMUSG00000063229"
[4] "ENSMUSG00000101959"

$`R-MMU-1296024`
[1] "ENSMUSG00000030247" "ENSMUSG00000040136" "ENSMUSG00000096146"

$`R-MMU-1369017`
[1] "ENSMUSG00000030247" "ENSMUSG00000030249" "ENSMUSG00000096146"

$`R-MMU-5678261`
[1] "ENSMUSG00000030249" "ENSMUSG00000096146"

$`R-MMU-9691566`
[1] "ENSMUSG00000030249" "ENSMUSG00000096146"

$`R-MMU-9649732`
[1] "ENSMUSG00000030265"

$`R-MMU-9653592`
[1] "ENSMUSG00000030265"

$`R-MMU-508189`
[1] "ENSMUSG00000030268"

$`R-MMU-70723`
[1] "ENSMUSG00000030268"

$`R-MMU-5679206`
[1] "ENSMUSG00000030269" "ENSMUSG00000034354"

$`R-MMU-110243`
[1] "ENSMUSG00000030271"

$`R-MMU-110244`
[1] "ENSMUSG00000030271"

$`R-MMU-110354`
[1] "ENSMUSG00000030271" "ENSMUSG00000035960"

$`R-MMU-5649701`
[1] "ENSMUSG00000030271" "ENSMUSG00000032298

[1] "ENSMUSG00000030895" "ENSMUSG00000052305" "ENSMUSG00000069919"
[4] "ENSMUSG00000073940"

$`R-MMU-2168886`
[1] "ENSMUSG00000030895"

$`R-MMU-2168897`
[1] "ENSMUSG00000030895" "ENSMUSG00000040249"

$`R-MMU-2230983`
[1] "ENSMUSG00000030895" "ENSMUSG00000040249"

$`R-MMU-5693347`
[1] "ENSMUSG00000030905"

$`R-MMU-70654`
[1] "ENSMUSG00000030934"

$`R-MMU-70666`
[1] "ENSMUSG00000030934"

$`R-MMU-8875039`
[1] "ENSMUSG00000030935"

$`R-MMU-159443`
[1] "ENSMUSG00000030945" "ENSMUSG00000033533"

$`R-MMU-159567`
[1] "ENSMUSG00000030945" "ENSMUSG00000030972" "ENSMUSG00000047026"

$`R-MMU-177157`
[1] "ENSMUSG00000030945" "ENSMUSG00000033533"

$`R-MMU-9749971`
[1] "ENSMUSG00000030945" "ENSMUSG00000030972" "ENSMUSG00000047026"

$`R-MMU-6788912`
[1] "ENSMUSG00000030946"

$`R-MMU-8932413`
[1] "ENSMUSG00000030960" "ENSMUSG00000037742"

$`R-MMU-189488`
[1] "ENSMUSG00000030979"

$`R-MMU-2089927`
[1] "ENSMUSG00000030987" "ENSMUSG00000032839"

$`R-MMU-2089943`
[1] "ENSMUSG00000030987" "ENSMUSG0000003283

[1] "ENSMUSG00000031558" "ENSMUSG00000045382" "ENSMUSG00000061353"

$`R-MMU-9010815`
[1] "ENSMUSG00000031558" "ENSMUSG00000056427"

$`R-MMU-9010872`
[1] "ENSMUSG00000031558" "ENSMUSG00000039952"

$`R-MMU-73596`
[1] "ENSMUSG00000031562"

$`R-MMU-437230`
[1] "ENSMUSG00000031565"

$`R-MMU-9756452`
[1] "ENSMUSG00000031565" "ENSMUSG00000032446" "ENSMUSG00000042821"
[4] "ENSMUSG00000062327"

$`R-NUL-420393`
[1] "ENSMUSG00000031565"

$`R-MMU-2029468`
[1] "ENSMUSG00000031570" "ENSMUSG00000070366"

$`R-MMU-196086`
[1] "ENSMUSG00000031574"

$`R-MMU-2534365`
[1] "ENSMUSG00000031576"

$`R-MMU-3323079`
[1] "ENSMUSG00000031584"

$`R-MMU-71682`
[1] "ENSMUSG00000031584"

$`R-MMU-194641`
[1] "ENSMUSG00000031604"

$`R-MMU-194669`
[1] "ENSMUSG00000031604"

$`R-MMU-2161779`
[1] "ENSMUSG00000031613"

$`R-MMU-9023968`
[1] "ENSMUSG00000031613"

$`R-MMU-9024766`
[1] "ENSMUSG00000031613"

$`R-MMU-9731931`
[1] "ENSMUSG00000031616"

$`R-MMU-202966`
[1] "ENSMUSG00000031626" "ENSMUSG00000031628"

$`R-MMU-201631`
[

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [40]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

 "R-MMU-5591040" "R-MMU-5604929"
[6237] "R-MMU-5607002" "R-MMU-8865605" "R-MMU-8865613" "R-MMU-8865728"
[6241] "R-MMU-451309"  "R-MMU-451310"  "R-MMU-8937419" "R-MMU-3928605"
[6245] "R-MMU-3928626" "R-MMU-9018747" "R-MMU-73810"   "R-MMU-73813"  
[6249] "R-MMU-73814"   "R-MMU-909718"  "R-MMU-909722"  "R-MMU-909724" 
[6253] "R-MMU-909729"  "R-MMU-909730"  "R-MMU-909732"  "R-MMU-912695" 
[6257] "R-MMU-9678737" "R-MMU-997309"  "R-MMU-997311"  "R-MMU-8854645"
[6261] "R-MMU-8986995" "R-MMU-8987040" "R-MMU-8987042" "R-MMU-8987105"
[6265] "R-MMU-8987132" "R-MMU-8987202" "R-MMU-8987223" "R-MMU-909720" 
[6269] "R-MMU-912685"  "R-MMU-9678935" "R-MMU-9696179" "R-MMU-3299691"
[6273] "R-MMU-3299753" "R-MMU-3697860" "R-MMU-3777112" "R-MMU-3780959"
[6277] "R-MMU-3780963" "R-MMU-8951723" "R-MMU-5653878" "R-MMU-5653886"
[6281] "R-MMU-432065"  "R-MMU-75889"   "R-MMU-435349"  "R-MMU-425577" 
[6285] "R-MMU-8876696" "R-MMU-8858346" "R-MMU-9835358" "R-MMU-187828" 
[6289] "R-MMU-9624668" "R-MMU-444523"  "R-MM

[6693] "R-MMU-2046097" "R-MMU-3000120" "R-MMU-8868402" "R-MMU-196964" 
[6697] "R-MMU-187020"  "R-MMU-8876789" "R-MMU-428007"  "R-MMU-5649883"
[6701] "R-MMU-5651723" "R-MMU-5651739" "R-MMU-8849882" "R-MMU-8979996"
[6705] "R-MMU-6789310" "R-MMU-1008248" "R-MMU-2586548" "R-MMU-2586557"
[6709] "R-MMU-2586558" "R-MMU-2671739" "R-MMU-2671740" "R-MMU-2671830"
[6713] "R-MMU-2671841" "R-MMU-2671852" "R-MMU-2672295" "R-MMU-5696482"
[6717] "R-MMU-6783681" "R-MMU-6784189" "R-MMU-6785898" "R-MMU-8950723"
[6721] "R-MMU-982775"  "R-MMU-982792"  "R-NUL-1067645" "R-NUL-879922" 
[6725] "R-NUL-8950050" "R-MMU-5693012" "R-MMU-390931"  "R-MMU-194698" 
[6729] "R-MMU-448629"  "R-MMU-2064406" "R-MMU-2064418" "R-MMU-2064421"
[6733] "R-MMU-351871"  "R-MMU-5693967" "R-MMU-9692206" "R-MMU-9692296"
[6737] "R-MMU-8869241" "R-MMU-8866268" "R-MMU-9638046" "R-MMU-1675773"
[6741] "R-MMU-1675810" "R-MMU-1676082" "R-MMU-1676134" "R-MMU-9837342"
[6745] "R-MMU-5696080" "R-MMU-1606583" "R-MMU-9647999" "R-MMU-2993802"
[6749]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[8329] "R-MMU-8851018" "R-MMU-5676599" "R-NUL-4084922" "R-NUL-4085088"
[8333] "R-MMU-6806967" "R-MMU-9635739" "R-MMU-9635743" "R-MMU-110360" 
[8337] "R-MMU-110375"  "R-MMU-111253"  "R-MMU-5649708" "R-MMU-5649725"
[8341] "R-MMU-5649854" "R-MMU-5651773" "R-MMU-5651789" "R-MMU-158744" 
[8345] "R-MMU-158747"  "R-MMU-158750"  "R-MMU-158756"  "R-MMU-158781" 
[8349] "R-MMU-158784"  "R-MMU-9724745" "R-MMU-9724753" "R-MMU-549112" 
[8353] "R-MMU-888614"  "R-MMU-198563"  "R-MMU-8870332" "R-MMU-204364" 
[8357] "R-MMU-428518"  "R-MMU-8986258" "R-MMU-9010815" "R-MMU-9010872"
[8361] "R-MMU-73596"   "R-MMU-437230"  "R-MMU-9756452" "R-NUL-420393" 
[8365] "R-MMU-2029468" "R-MMU-196086"  "R-MMU-2534365" "R-MMU-3323079"
[8369] "R-MMU-71682"   "R-MMU-194641"  "R-MMU-194669"  "R-MMU-2161779"
[8373] "R-MMU-9023968" "R-MMU-9024766" "R-MMU-9731931" "R-MMU-202966" 
[8377] "R-MMU-201631"  "R-MMU-201636"  "R-MMU-202917"  "R-MMU-202939" 
[8381] "R-MMU-202960"  "R-MMU-202967"  "R-MMU-211219"  "R-MMU-351877" 
[8385

[8789] "R-MMU-109862"  "R-MMU-109864"  "R-MMU-6784652" "R-MMU-8874201"
[8793] "R-MMU-8874202" "R-MMU-8874208" "R-MMU-373061"  "R-MMU-8867344"
[8797] "R-MMU-9840534" "R-MMU-6809495" "R-MMU-6809627" "R-MMU-5649681"
[8801] "R-MMU-5690023" "R-MMU-5577237" "R-MMU-9613264" "R-MMU-5626316"
[8805] "R-MMU-1605768" "R-MMU-4084994" "R-MMU-9611721" "R-MMU-9611751"
[8809] "R-MMU-6814137" "R-MMU-947673"  "R-MMU-372480"  "R-MMU-209738" 
[8813] "R-MMU-209944"  "R-MMU-8963851" "R-MMU-5696091" "R-MMU-3222116"
[8817] "R-MMU-1482598" "R-MMU-1482626" "R-MMU-389826"  "R-MMU-9733964"
[8821] "R-MMU-446189"  "R-MMU-426032"  "R-MMU-5662466" "R-MMU-388981" 
[8825] "R-MMU-8867186" "R-MMU-9007869" "R-MMU-264976"  "R-MMU-265010" 
[8829] "R-MMU-265153"  "R-MMU-109759"  "R-MMU-1678650" "R-MMU-1793182"
[8833] "R-MMU-6788611" "R-MMU-1638845" "R-MMU-888548"  "R-MMU-8875071"
[8837] "R-MMU-6810392" "R-MMU-6810410" "R-MMU-110351"  "R-MMU-110352" 
[8841] "R-MMU-110359"  "R-MMU-5649856" "R-MMU-110221"  "R-MMU-70361"  
[8845]

[9249] "R-NUL-9617912" "R-MMU-187246"  "R-MMU-189119"  "R-MMU-391942" 
[9253] "R-MMU-9726949" "R-MMU-1564169" "R-MMU-1602473" "R-MMU-389424" 
[9257] "R-MMU-9014672" "R-MMU-9014678" "R-MMU-162873"  "R-MMU-8878914"
[9261] "R-MMU-8858270" "R-MMU-5605262" "R-MMU-444620"  "R-MMU-9009950"
[9265] "R-MMU-8937744" "R-MMU-8937767" "R-MMU-914067"  "R-MMU-9604980"
[9269] "R-MMU-9606789" "R-MMU-8870352" "R-MMU-8870354" "R-MMU-419428" 
[9273] "R-MMU-3296301" "R-MMU-9830451" "R-MMU-975829"  "R-MMU-374787" 
[9277] "R-MMU-373086"  "R-MMU-211178"  "R-MMU-9626928" "R-NUL-9626944"
[9281] "R-MMU-170149"  "R-MMU-5671737" "R-MMU-5671749" "R-MMU-9029987"
[9285] "R-MMU-9733403" "R-MMU-419426"  "R-MMU-9696323" "R-MMU-374214" 
[9289] "R-MMU-417898"  "R-MMU-8850786" "R-MMU-400434"  "R-MMU-444202" 
[9293] "R-MMU-9732527" "R-MMU-9679029" "R-MMU-9724670" "R-MMU-8950248"
[9297] "R-MMU-8850777" "R-MMU-1462023" "R-MMU-1467269" "R-MMU-1471322"
[9301] "R-MMU-5638141" "R-MMU-964773"  "R-MMU-9013198" "R-MMU-428231" 
[9305]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [42]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122


In [43]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

  [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [51]  0  0  0  0  0  0  0  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 [76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[101] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30


In [44]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817613 SRR7817625 SRR3593523 SRR7817676 SRR4317609
ENSMUSG00000000001  15.678920  15.794204  14.802368  15.792070  15.257084
ENSMUSG00000000028   8.373881   8.010359   8.129061   8.402368   8.704348
ENSMUSG00000000049  20.478337  20.313465  20.754121  20.038138  20.600894
ENSMUSG00000000058   8.871176   8.801971   9.197780   9.541066   9.859166
ENSMUSG00000000085  10.969441  11.280353  11.256788  10.338241  10.352565
ENSMUSG00000000088  16.818565  16.528197  15.272027  16.605132  14.719661
ENSMUSG00000000120  10.257422   9.767201  10.096116   8.761923   9.224136
ENSMUSG00000000126   7.735798   8.169543   8.368686   8.206634   7.727472
ENSMUSG00000000127  10.899682   9.940755  10.029376   9.698646  10.119518
ENSMUSG00000000142  10.346006  10.242311  12.400591   9.584448  11.170299
ENSMUSG00000000149  16.031783  15.949346  15.781778  16.193725  15.527097
ENSMUSG00000000154  15.680051  15.417805  15.948595  15.693681  15.571908
ENSMUSG00000000168  15.128186  14.5369

ENSMUSG00000001105  13.834652  14.063923  13.369337  13.678283  13.657741
ENSMUSG00000001119   9.902588   9.508806  11.511828  10.162278   9.767685
ENSMUSG00000001123  18.686733  18.526351  19.018877  17.513161  18.548614
ENSMUSG00000001127  15.249684  15.164983  15.519729  15.190110  15.124798
ENSMUSG00000001128  12.080440  11.070983  13.812111  12.348856  11.827959
ENSMUSG00000001131   6.312395   7.167866   7.589523   6.336819   7.187954
ENSMUSG00000001143  12.159266  12.068519  12.454888  11.853980  11.508504
ENSMUSG00000001155  19.038346  18.960249  18.460769  18.734232  18.781679
ENSMUSG00000001158  11.667119  11.678295  11.581374  11.355193  11.673604
ENSMUSG00000001173  10.811874  10.310998  10.865944  10.101524  10.886458
ENSMUSG00000001175  17.459418  17.238606  16.316359  17.457228  16.141188
ENSMUSG00000001211  17.138803  17.098830  17.310386  16.922639  16.974909
ENSMUSG00000001225   7.422874   6.355630   6.232838   6.336819   7.187954
ENSMUSG00000001228   7.888018   7.8172

ENSMUSG00000002249  11.292581  11.617850  12.022693  11.316930  10.686930
ENSMUSG00000002250  14.102530  11.832726  12.065508  10.957526  10.379301
ENSMUSG00000002257   7.942434   7.817265   7.798773   7.709799   7.038213
ENSMUSG00000002279  13.806844  13.673847  13.981500  13.976735  13.426144
ENSMUSG00000002289  14.176110  14.454860  16.638793  14.633083  16.542227
ENSMUSG00000002297   9.300576   9.226524   8.600207   8.717390   9.286930
ENSMUSG00000002307  11.692069  11.787139  12.354117  11.072413  11.510136
ENSMUSG00000002308   9.788935   9.293663   9.505807   9.194806   8.880946
ENSMUSG00000002324   9.041932   8.870210   9.224417   8.873165   8.688120
ENSMUSG00000002325  13.893143  13.792341  14.660869  13.278001  13.582999
ENSMUSG00000002326  12.065886  12.129189  11.653973  11.505039  11.557026
ENSMUSG00000002346  15.944946  15.790767  17.065235  15.725146  15.538659
ENSMUSG00000002365  14.171765  13.910420  13.946860  14.070223  13.556913
ENSMUSG00000002379  14.329350  14.0317

ENSMUSG00000003379   7.548325   7.817265   8.878969   7.837426   7.846889
ENSMUSG00000003402  16.137511  15.872430  16.521023  16.009264  15.817103
ENSMUSG00000003411   7.735798   7.817265   7.637264   7.742548   7.849076
ENSMUSG00000003418   6.312395   6.355630   7.178062   7.123154   7.430730
ENSMUSG00000003421  12.966094  12.928049  12.648176  12.433760  12.140279
ENSMUSG00000003429  14.593308  14.479566  15.437480  14.207576  15.417206
ENSMUSG00000003435  14.877766  15.150117  15.170636  15.397468  14.686444
ENSMUSG00000003436   6.312395   6.355630   6.232838   6.336819   6.364052
ENSMUSG00000003437  13.368185  13.676418  13.422618  13.431222  12.949344
ENSMUSG00000003444   9.745042   9.342889   9.779391  10.059250   9.849863
ENSMUSG00000003452   6.312395   6.355630   7.056556   7.123154   7.734980
ENSMUSG00000003458  14.277818  14.192523  14.708459  14.322657  14.460885
ENSMUSG00000003464  15.855317  15.606630  16.507704  15.686295  16.196529
ENSMUSG00000003476   7.647951   7.1678

ENSMUSG00000004654   6.312395   6.355630   7.056556   6.336819   7.187954
ENSMUSG00000004655  14.819536  14.798393  15.765014  14.727380  16.044452
ENSMUSG00000004665   9.633870   9.867161  10.025404  10.239755   9.047067
ENSMUSG00000004667  12.829386  12.711156  13.513276  12.834857  12.344046
ENSMUSG00000004668   6.312395   6.355630   6.232838   6.336819   6.364052
ENSMUSG00000004677  13.485695  13.751344  13.427651  13.375644  13.042758
ENSMUSG00000004709   7.065403   6.355630   7.421220   7.123154   7.734980
ENSMUSG00000004730  10.982535  11.014651  11.963230  11.134930  11.375573
ENSMUSG00000004768  11.122703  11.415269   9.940275  10.415418  10.844804
ENSMUSG00000004771  14.784384  14.700623  13.846547  14.711776  13.935375
ENSMUSG00000004788  12.314744  12.610442  12.150095  12.321622  11.435672
ENSMUSG00000004789  17.258729  16.963554  16.897006  17.218390  16.944020
ENSMUSG00000004791   6.312395   6.355630   6.232838   7.123154   6.364052
ENSMUSG00000004815  14.178498  14.0308

...

ENSMUSG00000008090  11.965587  11.457238  11.685998  11.432766  12.170034
ENSMUSG00000008167  15.162433  15.779797  14.494625  14.945557  15.114398
ENSMUSG00000008206  10.059463  10.273058  10.087852  10.305159  10.155310
ENSMUSG00000008301  12.484766  11.398898  12.105399  11.694022  12.072207
ENSMUSG00000008305  13.741188  13.485797  13.512719  13.819606  13.808115
ENSMUSG00000008333  11.013961  11.008070  11.652029  10.870040  11.395968
ENSMUSG00000008348  20.043508  19.768437  19.139678  19.490219  19.895750
ENSMUSG00000008373  12.215769  12.379829  11.973921  12.305317  12.563960
ENSMUSG00000008435  10.779240  11.850824  10.579320  11.673166  11.718078
ENSMUSG00000008438   7.139293   6.964996   6.979401   6.266965   6.317624
ENSMUSG00000008461  10.649254   9.926635   9.408723  11.174161  10.897392
ENSMUSG00000008475  15.387969  15.592232  15.419402  15.423126  15.718553
ENSMUSG00000008496   9.954129   9.678561   8.995186  11.672611   9.633635
ENSMUSG00000008540  20.190498  19.6902

ENSMUSG00000011463   6.262078   6.269604   6.275767   6.266965   6.317624
ENSMUSG00000011658  11.907643  12.010139  11.259353  12.004654  12.125275
ENSMUSG00000011751   7.251898   7.712987   7.178054   7.153126   7.757312
ENSMUSG00000011752  14.030788  14.906279  13.148139  13.748446  14.500746
ENSMUSG00000011837  10.720102  10.462228   9.902223  10.330973  10.569606
ENSMUSG00000011877  11.702790  12.690784  10.510662  13.211363  12.166511
ENSMUSG00000011884  12.647452  13.053748  10.285860  13.828527  13.022594
ENSMUSG00000011958  13.932479  13.641391  14.252266  13.639408  13.817615
ENSMUSG00000011960  13.089635  13.282902  13.688087  13.546939  13.495211
ENSMUSG00000012114  12.977989  13.757774  12.503990  13.928632  13.723674
ENSMUSG00000012117  12.939247  13.557647  13.434977  13.154496  13.197229
ENSMUSG00000012126  10.375721  10.672947   8.568683  10.479518  11.249487
ENSMUSG00000012187   7.270534   7.160597   7.510178   7.461141   7.755617
ENSMUSG00000012282   6.262078   6.2696

ENSMUSG00000001773   6.289001   6.311837
ENSMUSG00000001786  13.375168  12.764904
ENSMUSG00000001794  15.931661  16.171359
ENSMUSG00000001804   6.289001   6.311837
ENSMUSG00000001827   7.703293   7.812757
ENSMUSG00000001833  14.961473  15.203882
ENSMUSG00000001847  15.850372  15.683200
ENSMUSG00000001855  12.331601  12.101097
ENSMUSG00000001865   7.355943   6.311837
ENSMUSG00000001870  10.431133  10.169122
ENSMUSG00000001891  18.717007  17.858985
ENSMUSG00000001901   6.289001   6.311837
ENSMUSG00000001918  14.444479  14.693938
ENSMUSG00000001924  16.315726  16.834710
ENSMUSG00000001930  11.610192  11.631107
ENSMUSG00000001946  11.663493  10.688878
ENSMUSG00000001962  12.013642  12.597857
ENSMUSG00000001964  11.673114  11.386000
ENSMUSG00000001983  11.793427  12.332222
ENSMUSG00000001985   6.289001   6.311837
ENSMUSG00000001986   7.893759   8.583681
ENSMUSG00000001998  11.269870  10.985876
ENSMUSG00000001999  12.167024  11.580424
ENSMUSG00000002010  14.358211  14.430882
ENSMUSG000000020

ENSMUSG00000003848  12.097076  12.265427
ENSMUSG00000003849  16.660757  16.099477
ENSMUSG00000003863   7.055706   7.064088
ENSMUSG00000003865   9.401155   9.712611
ENSMUSG00000003872   6.289001   6.311837
ENSMUSG00000003873  13.992457  13.016435
ENSMUSG00000003882  10.922862   9.503622
ENSMUSG00000003923  14.024896  14.112296
ENSMUSG00000003934   7.559497   7.730181
ENSMUSG00000003970  17.761323  17.361249
ENSMUSG00000003974   6.289001   6.311837
ENSMUSG00000004018  10.028890   9.696336
ENSMUSG00000004032   9.545223  10.019064
ENSMUSG00000004035  10.679176  12.231256
ENSMUSG00000004038  20.084194  19.428873
ENSMUSG00000004040  14.796887  15.768518
ENSMUSG00000004043  10.705338  10.289291
ENSMUSG00000004044  12.603069  11.628410
ENSMUSG00000004054  13.388928  13.103532
ENSMUSG00000004056  14.631108  14.215466
ENSMUSG00000004070  13.942097  13.918508
ENSMUSG00000004096  13.973956  13.693321
ENSMUSG00000004098   9.656812   9.614132
ENSMUSG00000004099  12.040205  12.119842
ENSMUSG000000041

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

       Resample1
  [1,]         1
  [2,]         2
  [3,]         3
  [4,]         4
  [5,]         5
  [6,]         6
  [7,]         7
  [8,]         8
  [9,]         9
 [10,]        10
 [11,]        11
 [12,]        12
 [13,]        13
 [14,]        14
 [15,]        15
 [16,]        16
 [17,]        17
 [18,]        18
 [19,]        19
 [20,]        20
 [21,]        21
 [22,]        22
 [23,]        23
 [24,]        24
 [25,]        25
 [26,]        26
 [27,]        27
 [28,]        28
 [29,]        29
 [30,]        30
 [31,]        31
 [32,]        32
 [33,]        33
 [34,]        34
 [35,]        35
 [36,]        36
 [37,]        37
 [38,]        38
 [39,]        39
 [40,]        40
 [41,]        41
 [42,]        42
 [43,]        43
 [44,]        44
 [45,]        45
 [46,]        46
 [47,]        47
 [48,]        48
 [49,]        49
 [50,]        50
 [51,]        51
 [52,]        52
 [53,]        53
 [54,]        54
 [55,]        55
 [56,]        56
 [57,]        57
 [58,]        

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(head(vst.count.mtx.train))

                   SRR7817613 SRR7817625 SRR3593523 SRR7817676 SRR4317609
ENSMUSG00000000001  15.678920  15.794204  14.802368  15.792070  15.257084
ENSMUSG00000000028   8.373881   8.010359   8.129061   8.402368   8.704348
ENSMUSG00000000049  20.478337  20.313465  20.754121  20.038138  20.600894
ENSMUSG00000000058   8.871176   8.801971   9.197780   9.541066   9.859166
ENSMUSG00000000085  10.969441  11.280353  11.256788  10.338241  10.352565
ENSMUSG00000000088  16.818565  16.528197  15.272027  16.605132  14.719661
                   SRR7817660 SRR7817664 SRR7817648 SRR7817698 SRR7817651
ENSMUSG00000000001  15.856177  15.530593  15.446439  15.544915  15.813822
ENSMUSG00000000028   8.303532   8.032180   8.124688   7.973644   8.602236
ENSMUSG00000000049  20.164105  20.345501  20.527072  20.309176  20.190658
ENSMUSG00000000058   8.975952   9.078611   8.706991   8.868303   8.983531
ENSMUSG00000000085  11.046926  10.509282  10.980802  10.712587  10.772729
ENSMUSG00000000088  16.217968  16.8491

In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_90.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
 [1]   1  28  29  30  43  45  52  63  69  75 102 113 115

$Fold02
 [1]   9  35  39  41  48  61  85  93  94 106 108 120

$Fold03
 [1]   2   4   6  32  33  42  71  72  81  84  90 107 114

$Fold04
 [1]   5   7  36  44  58  60  74  76  87  92 109 122

$Fold05
 [1]  14  15  22  34  54  57  65  67  77  88 101 118

$Fold06
 [1]  11  18  25  40  50  55  66  68  73  78 100 112

$Fold07
 [1]  19  21  23  37  51  53  82  95  99 103 110 121

$Fold08
 [1]  3 16 17 26 49 56 64 79 80 91 96 97

$Fold09
 [1]   8  10  20  27  38  47  62  83  98 105 111 116

$Fold10
 [1]  12  13  24  31  46  59  70  86  89 104 117 119



In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1]  0 30


In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_90.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_90.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_90.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951517 371.3   13066828 697.9 13066828 697.9
Vcells 17134337 130.8   31507692 240.4 30243165 230.8


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_90.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_90.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

 [1]   1  28  29  30  43  45  52  63  69  75 102 113 115
[1] "----------"
 [1]   9  35  39  41  48  61  85  93  94 106 108 120
[1] "----------"
 [1]   2   4   6  32  33  42  71  72  81  84  90 107 114
[1] "----------"
 [1]   5   7  36  44  58  60  74  76  87  92 109 122
[1] "----------"
 [1]  14  15  22  34  54  57  65  67  77  88 101 118
[1] "----------"
 [1]  11  18  25  40  50  55  66  68  73  78 100 112
[1] "----------"
 [1]  19  21  23  37  51  53  82  95  99 103 110 121
[1] "----------"
 [1]  3 16 17 26 49 56 64 79 80 91 96 97
[1] "----------"
 [1]   8  10  20  27  38  47  62  83  98 105 111 116
[1] "----------"
 [1]  12  13  24  31  46  59  70  86  89 104 117 119
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  1 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.68932301456088 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.100485494053422 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.112693140858451 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  0.96648199

[1] "Last RXN_ID =  R-MMU-9034539 : Last ARI =  -0.0150716185053293 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  1 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.969047619047619 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  1 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  0.966481994459834 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  0.86849

[1] "Last RXN_ID =  R-MMU-174757 : Last ARI =  1 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-187506 : Last ARI =  0.188088201960899 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.1 % remaining..."
[1] "Last RXN_ID =  R-NUL-9625422 : Last ARI =  0.582131780192722 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79 % remaining..."
[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  1 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =  0.830030119157543 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848484 : Last ARI =  0.9355296135

[1] "Last RXN_ID =  R-MMU-9623032 : Last ARI =  0.142256035648521 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69 % remaining..."
[1] "Last RXN_ID =  R-MMU-1592398 : Last ARI =  0.966481994459834 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  68.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5683888 : Last ARI =  0.126298065528523 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  68.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-195275 : Last ARI =  0.966481994459834 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  68.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5229343 : Last ARI =  0.966481994459834 : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  68.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-5579023 : Last ARI =  0.314710730943418 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  68.4 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-4341016 : Last ARI =  0.736987204854241 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-4568848 : Last ARI =  0.841572047527726 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-4615839 : Last ARI =  0.932963988919668 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-4655356 : Last ARI =  0.569227966959829 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-4755419 : Last ARI =  0.845489868836852 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-1675994 : Last ARI =  0.848849144218288 : Last ECOUNT =  7 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  58.3 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-77613 : Last ARI =  0.190912661170265 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8963734 : Last ARI =  0.725233364598654 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9708430 : Last ARI =  0.0546455545070503 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-190427 : Last ARI =  0.966481994459834 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-168053 : Last ARI =  0.777223761157279 : Last ECOUNT =  8 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8871376 : Last ARI =  0.273470602958138 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  48.1 % remaining..."
[1] "Last RXN

[1] "Last RXN_ID =  R-MMU-2473594 : Last ARI =  0.899445983379501 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  38.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-446648 : Last ARI =  0.966481994459834 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  38.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848314 : Last ARI =  0.902011607967287 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  38.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-879221 : Last ARI =  0.902011607967287 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  38.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-3371531 : Last ARI =  0.899445983379501 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  38 % remaining..."
[1] "Last RXN_ID =  R-MMU-6783955 : Last ARI =  0.206023225586431 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  37.9 % remaining..."
[1] "Last RXN_

[1] "Last RXN_ID =  R-MMU-4084912 : Last ARI =  0.93956043956044 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  28.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-421140 : Last ARI =  0.935529613507453 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  28.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9627350 : Last ARI =  0.606696211608629 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  28 % remaining..."
[1] "Last RXN_ID =  R-NUL-2169043 : Last ARI =  0.209564798278318 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  27.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-448702 : Last ARI =  0.454813399896502 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  27.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-74872 : Last ARI =  0.935651954934142 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  27.7 % remaining..."
[1] "Last RXN_ID 

[1] "Last RXN_ID =  R-MMU-8847534 : Last ARI =  0.25899112510975 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-139854 : Last ARI =  0.966481994459834 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18 % remaining..."
[1] "Last RXN_ID =  R-MMU-198983 : Last ARI =  0.509000559238425 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.9 % remaining..."
[1] "Last RXN_ID =  R-NUL-2128982 : Last ARI =  -0.0186123687814673 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9757010 : Last ARI =  0.966481994459834 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9653592 : Last ARI =  -0.00496448532749509 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.6 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-NUL-844613 : Last ARI =  0.217377992788227 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9629360 : Last ARI =  0.326003470213997 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-6809624 : Last ARI =  0.263000886120983 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-192312 : Last ARI =  0.932963988919668 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8879117 : Last ARI =  0.593646227579746 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139909 : Last ARI =  0.0624324928027466 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.4 % remaining..."
[1] "Last RXN_ID =

In [28]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_90.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_90.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_90.Rds", sep = ""))

In [29]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_90.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  9.664820e-01     14
R-MMU-170666.30.00   R-MMU-170671        0  9.664820e-01     14
R-MMU-170671.0.00    R-MMU-170674        0  9.664820e-01     14
R-MMU-170671.30.00   R-MMU-170686        0  9.664820e-01     14
R-MMU-170674.0.00    R-MMU-380073        0  1.000000e+00    182
R-MMU-170674.30.00   R-MMU-392187        0  1.000000e+00     22
R-MMU-170686.0.00    R-MMU-392195        0  1.000000e+00     22
R-MMU-170686.30.00   R-MMU-392202        0  1.000000e+00     22
R-MMU-380073.0.00    R-MMU-749454        0  1.000000e+00    178
R-MMU-380073.30.00   R-MMU-749456        0  1.000000e+00    178
R-MMU-392187.0.00    R-MMU-751001        0  1.000000e+00     21
R-MMU-392187.30.00  R-MMU-8964229        0  1.000000e+00      5
R-MMU-392195.0.00   R-MMU-8964242        0  1.000000e+00      5
R-MMU-392195.30.00  R-MMU-8964329        0  1.000000e+00      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-5654224.30.00 R-MMU-9011082        0  7.159905e-01      2
R-MMU-5654392.0.00  R-MMU-9011144        0  6.552103e-02      3
R-MMU-5654392.30.00 R-MMU-9011146        0  6.552103e-02      3
R-MMU-5654397.0.00  R-MMU-9753028        0  7.201437e-01      5
R-MMU-5654397.30.00 R-MMU-9753126        0  7.201437e-01      5
R-MMU-5654399.0.00   R-MMU-514604        0  4.666592e-01      2
R-MMU-5654399.30.00  R-MMU-975389        0  6.124817e-01      4
R-MMU-5654402.0.00  R-MMU-1169394        0  9.690476e-01      5
R-MMU-5654402.30.00 R-MMU-1169398        0  8.094062e-01      5
R-MMU-5654404.0.00  R-MMU-1169402        0  8.710592e-01      6
R-MMU-5654404.30.00 R-MMU-1169403        0  6.031975e-01      4
R-MMU-5654406.0.00  R-MMU-1169405        0  9.020116e-01      5
R-MMU-5654406.30.00 R-MMU-1169406        0  9.664820e-01     12
R-MMU-5654407.0.00  R-MMU-1678842        0  1.000000e+00     13
R-MMU-5654407.30.00 R-MMU-1678843        0  8.710592e-01      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-9011082.30.00 R-MMU-9692139        0  1.500302e-01      2
R-MMU-9011144.0.00  R-MMU-9692145        0  7.477368e-01      2
R-MMU-9011144.30.00 R-MMU-9692156        0  3.566347e-01      4
R-MMU-9011146.0.00  R-MMU-9692157        0  3.566347e-01      4
R-MMU-9011146.30.00 R-MMU-9692268        0  2.885244e-01      2
R-MMU-9753028.0.00   R-MMU-450434        0  2.190491e-01      1
R-MMU-9753028.30.00  R-MMU-450466        0  9.690476e-01     45
R-MMU-9753126.0.00   R-MMU-450551        0  9.045772e-01      7
R-MMU-9753126.30.00  R-MMU-450580        0  9.664820e-01     12
R-MMU-514604.0.00     R-MMU-72103        0  9.329640e-01     43
R-MMU-514604.30.00  R-MMU-9770119        0  1.000000e+00     69
R-MMU-975389.0.00   R-MMU-9770141        0  1.000000e+00    123
R-MMU-975389.30.00  R-MMU-9770142        0  1.000000e+00    132
R-MMU-1169394.0.00  R-MMU-9770145        0  1.000000e+00    145
R-MMU-1169394.30.00 R-MMU-9770236        0  1.000000e+00    119
R-MMU-1169398.0.00  R-MMU-6798748       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 R-MMU-1445150        0  9.664820e-01      3
R-MMU-5693542.30.00  R-MMU-162352        0  2.848640e-01      1
R-MMU-5693564.0.00   R-MMU-198599        0  9.086081e-01      4
R-MMU-5693564.30.00  R-MMU-198609        0  9.664820e-01      4
R-MMU-5693580.0.00   R-MMU-198611        0  9.664820e-01      4
R-MMU-5693580.30.00  R-MMU-198613        0  1.000000e+00      5
R-MMU-5693593.0.00   R-MMU-198621        0  9.664820e-01      4
R-MMU-5693593.30.00  R-MMU-198640        0  1.000000e+00     10
R-MMU-6782138.0.00   R-MMU-199298        0  9.355296e-01      4
R-MMU-6782138.30.00  R-MMU-199299        0  8.994460e-01      7
R-MMU-6782141.0.00   R-MMU-199425        0  1.000000e+00      5
R-MMU-6782141.30.00  R-MMU-199443        0  9.664820e-01      5
R-MMU-6782204.0.00   R-MMU-199839        0  9.355296e-01      4
R-MMU-6782204.30.00  R-MMU-199863        0  9.355296e-01      4
R-MMU-6782208.0.00   R-MMU-200143        0  9.355296e-01      4
R-MMU-6782208.30.00  R-MMU-211164        0  8.994460e-01   

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        0  7.843788e-01      3
R-NUL-9617903.0.00  R-MMU-5692764        0  8.718529e-01      4
R-NUL-9617903.30.00  R-MMU-191366        0  4.928608e-01      1
R-NUL-9617912.0.00   R-NUL-936401        0  2.649895e-01      1
R-NUL-9617912.30.00  R-MMU-744230        0  3.995018e-01      1
R-MMU-187246.0.00    R-MMU-744231        0  3.995018e-01      1
R-MMU-187246.30.00  R-MMU-9792608        0  3.995018e-01      1
R-MMU-189119.0.00   R-MMU-5694563        0  4.867071e-02      1
R-MMU-189119.30.00  R-MMU-1482543        0  4.641701e-01      1
R-MMU-391942.0.00    R-MMU-426043        0  4.641701e-01      1
R-MMU-391942.30.00  R-MMU-2872452        0  4.830029e-01      1
R-MMU-9726949.0.00  R-MMU-2408515        0 -7.014300e-03      1
R-MMU-9726949.30.00 R-MMU-9014420        0  5.433314e-01      1
R-MMU-1564169.0.00  R-MMU-9714480        0  5.433314e-01      1
R-MMU-1564169.30.00 R-MMU-9714483        0  5.433314e-01      1
R-MMU-1602473.0.00  R-MMU-5693681        0  8.711816e-01      2
R-MMU-160

R-MMU-8850854.30.00 R-MMU-8932275        0  4.806588e-02      2
R-MMU-8954468.0.00  R-NUL-1236944        0  1.184738e-01      1
R-MMU-8954468.30.00 R-MMU-9695890        0  8.409005e-01      2
R-MMU-70542.0.00     R-MMU-425983        0  3.089525e-01      2
R-MMU-70542.30.00    R-MMU-426015        0  6.995877e-02      2
R-MMU-6804402.0.00   R-MMU-390886        0  4.713168e-01      1
R-MMU-6804402.30.00 R-MMU-8869206        0 -3.938845e-02      1
R-MMU-418172.0.00    R-MMU-196955        0  5.139584e-02      1
R-MMU-418172.30.00   R-MMU-391211        0  5.955678e-01      5
R-MMU-9708866.0.00   R-MMU-435366        0  3.171165e-01      1
R-MMU-9708866.30.00 R-MMU-8938300        0  2.593302e-02      1
R-MMU-9006144.0.00   R-MMU-196857        0  5.566063e-01      1
R-MMU-9006144.30.00 R-MMU-5696101        0  6.035164e-01      1
R-MMU-5336430.0.00  R-MMU-1474146        0  1.000000e+00      2
R-MMU-5336430.30.00 R-MMU-1474158        0  1.000000e+00      2
R-MMU-5687126.0.00  R-MMU-1296037       

R-MMU-9011343.30.00  R-MMU-170825        0  9.329640e-01      1
R-MMU-9816416.0.00  R-MMU-9013435        0 -2.699116e-02      1
R-MMU-9816416.30.00 R-MMU-9013452        0 -2.699116e-02      1
R-MMU-9820700.0.00  R-MMU-9693207        0 -2.699116e-02      1
R-MMU-9820700.30.00 R-MMU-9693243        0 -2.699116e-02      1
R-MMU-1296035.0.00  R-NUL-2426638        0  1.210293e-02      1
R-MMU-1296035.30.00  R-NUL-420389        0  1.210293e-02      1
R-MMU-9013021.0.00   R-MMU-191380        0  3.585472e-01      1
R-MMU-9013021.30.00 R-MMU-5675947        0  5.970343e-01      2
R-MMU-9690494.0.00  R-MMU-5603114        0  1.891580e-01      2
R-MMU-9690494.30.00 R-MMU-5695964        0  9.021339e-01      1
R-NUL-4649029.0.00  R-MMU-1606789        0  2.075426e-01      1
R-NUL-4649029.30.00 R-MMU-1793207        0  2.075426e-01      1
R-MMU-1482861.0.00   R-MMU-389821        0  5.593495e-01      1
R-MMU-1482861.30.00 R-MMU-6797653        0  2.420292e-01      1
R-MMU-1482867.0.00  R-MMU-8849969       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_90.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_90.txt",sep=""), header=FALSE)
print(sid)

            V1
1   SRR7817613
2   SRR7817625
3   SRR3593523
4   SRR7817676
5   SRR4317609
6   SRR7817660
7   SRR7817664
8   SRR7817648
9   SRR7817698
10  SRR7817651
11  SRR4317610
12  SRR7817611
13  SRR7817652
14  SRR7817612
15  SRR7817638
16  SRR1636586
17  SRR7817637
18  SRR7817615
19  SRR1636593
20  SRR7817674
21  SRR4317611
22  SRR7817639
23  SRR7817673
24  SRR7817636
25  SRR7817696
26  SRR7817688
27  SRR7817661
28  SRR7817614
29  SRR7817675
30  SRR4317612
31  SRR7817685
32  SRR1636594
33  SRR7817616
34  SRR7817663
35  SRR7817697
36  SRR7817699
37  SRR4317608
38  SRR7817628
39  SRR7817647
40  SRR7817662
41  SRR7817649
42  SRR7817683
43  SRR7817650
44  SRR4317607
45  SRR1636592
46  SRR1636590
47  SRR7817672
48  SRR1636591
49  SRR7817640
50  SRR7817623
51  SRR3593526
52  SRR1636588
53  SRR3593525
54  SRR7817626
55  SRR7817684
56  SRR7817624
57  SRR7817686
58  SRR7817700
59  SRR3593527
60  SRR7817659
61  SRR3593524
62  SRR7817620
63  SRR7817646
64  SRR7817704
65  SRR7817642
66  SRR431

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  
     colors = c("red", "green"),  
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_90.html", selfcontained = TRUE)